In [ ]:
# Install databento if needed
# %pip install databento

In [1]:
import databento as db
from dotenv import load_dotenv
import os
from datetime import datetime, timedelta
import pandas as pd

# Load environment variables
load_dotenv('../data_pipeline/.env')

# Initialize Databento client
api_key = os.getenv('DATA_BENTO_API_KEY')
if not api_key:
    raise ValueError("DATA_BENTO_API_KEY not found")

client = db.Historical(key=api_key)
print("Databento client initialized")

Databento client initialized


In [34]:
# Test available datasets
datasets = client.metadata.list_datasets()
print("Available datasets:")
for ds in datasets:  # Show all
    print(f"- {ds}")

# Look for options datasets (OPRA is options)
options_datasets = [ds for ds in datasets if 'opra' in ds.lower() or 'option' in ds.lower()]
print(f"\nOptions datasets: {options_datasets}")

Available datasets:
- ARCX.PILLAR
- BATS.PITCH
- BATY.PITCH
- DBEQ.BASIC
- EDGA.PITCH
- EDGX.PITCH
- EPRL.DOM
- EQUS.MINI
- EQUS.SUMMARY
- GLBX.MDP3
- IEXG.TOPS
- IFEU.IMPACT
- IFLL.IMPACT
- IFUS.IMPACT
- MEMX.MEMOIR
- NDEX.IMPACT
- OPRA.PILLAR
- XASE.PILLAR
- XBOS.ITCH
- XCHI.PILLAR
- XCIS.TRADESBBO
- XEEE.EOBI
- XEUR.EOBI
- XNAS.BASIC
- XNAS.ITCH
- XNYS.PILLAR
- XPSX.ITCH

Options datasets: ['OPRA.PILLAR']


In [14]:
# Test SPX options data
# SPX options are traded on CBOE via OPRA
# Dataset is 'OPRA.PILLAR'
# NOTE: Live data license required for data after 2025-11-25T14:30:00Z

try:
    # Get historical SPX options data (before license cutoff)
    end_date = datetime(2025, 11, 25, 14, 30, 0)  # Before license requirement
    start_date = datetime(2025, 11, 24, 14, 30, 0)  # One day earlier
    
    # Try without symbols first to see what's available
    print(f"Querying from {start_date} to {end_date}")
    data = client.timeseries.get_range(
        dataset='OPRA.PILLAR',
        schema='trades',
        start=start_date,
        end=end_date,
        limit=100  # Small limit to see sample
    )
    
    df = data.to_df()
    print(f"OPRA data shape: {df.shape}")
    if not df.empty:
        print("OPRA data available!")
        print(df.head())
        
        # Analyze the data
        unique_symbols = df['symbol'].unique()
        print(f"Unique option symbols: {len(unique_symbols)}")
        print("Sample symbols:", unique_symbols[:10])
        
        # Look for SPX options
        spx_symbols = [s for s in unique_symbols if s.startswith('SPX')]
        print(f"SPX option symbols found: {len(spx_symbols)}")
        if spx_symbols:
            print("Sample SPX symbols:", spx_symbols[:5])
        
    else:
        print("No OPRA data found")
        
except Exception as e:
    print(f"Error fetching OPRA data: {e}")
    
# Try with some common option symbols
try:
    print("\nTrying with common SPX option symbols...")
    # Common SPX option format: SPXYYMMDD[C|P]XXXXX
    # Let's try some recent expirations
    test_symbols = ['SPX251125C04500', 'SPX251125P04500']  # Nov 25 expiration, 4500 strike
    
    data_test = client.timeseries.get_range(
        dataset='OPRA.PILLAR',
        symbols=test_symbols,
        schema='trades',
        start=start_date,
        end=end_date,
        limit=10
    )
    
    df_test = data_test.to_df()
    print(f"Test symbols data shape: {df_test.shape}")
    if not df_test.empty:
        print("Test symbols data found!")
        print(df_test.head())
    
except Exception as e:
    print(f"Test symbols query failed: {e}")

Querying from 2025-11-24 14:30:00 to 2025-11-25 14:30:00
OPRA data shape: (100, 13)
OPRA data available!
                                                               ts_event  \
ts_recv                                                                   
2025-11-24 14:30:00.002383233+00:00 2025-11-24 14:30:00.002181954+00:00   
2025-11-24 14:30:00.002428250+00:00 2025-11-24 14:30:00.002227076+00:00   
2025-11-24 14:30:00.002463080+00:00 2025-11-24 14:30:00.002262114+00:00   
2025-11-24 14:30:00.002498190+00:00 2025-11-24 14:30:00.002297201+00:00   
2025-11-24 14:30:00.002538338+00:00 2025-11-24 14:30:00.002337329+00:00   

                                     rtype  publisher_id  instrument_id  \
ts_recv                                                                   
2025-11-24 14:30:00.002383233+00:00      0            35     1207959668   
2025-11-24 14:30:00.002428250+00:00      0            35     1207960281   
2025-11-24 14:30:00.002463080+00:00      0            35     12079608

In [16]:
# Try to get symbol definitions
try:
    print("Getting option symbol definitions from midnight...")
    # Start from midnight for complete definitions
    def_start = datetime(2025, 11, 24, 0, 0, 0)
    def_end = datetime(2025, 11, 25, 0, 0, 0)
    
    definitions = client.timeseries.get_range(
        dataset='OPRA.PILLAR',
        schema='definition',
        start=def_start,
        end=def_end,
        limit=100  # Smaller limit
    )
    
    df_def = definitions.to_df()
    print(f"Definitions shape: {df_def.shape}")
    if not df_def.empty:
        print("Sample definitions:")
        print(df_def.head())
        
        # Look for SPX options
        if 'symbol' in df_def.columns:
            spx_defs = df_def[df_def['symbol'].str.startswith('SPX', na=False)]
            print(f"SPX definitions found: {len(spx_defs)}")
            if not spx_defs.empty:
                print("Sample SPX definitions:")
                print(spx_defs[['symbol', 'instrument_id']].head())
    
except Exception as e:
    print(f"Definitions query failed: {e}")

Getting option symbol definitions from midnight...
Definitions shape: (100, 64)
Sample definitions:
                                                               ts_event  \
ts_recv                                                                   
2025-11-24 11:30:00.510234758+00:00 2025-11-24 11:30:00.510033564+00:00   
2025-11-24 11:30:00.510250326+00:00 2025-11-24 11:30:00.510048361+00:00   
2025-11-24 11:30:00.510278519+00:00 2025-11-24 11:30:00.510077360+00:00   
2025-11-24 11:30:00.510294254+00:00 2025-11-24 11:30:00.510092446+00:00   
2025-11-24 11:30:00.510815996+00:00 2025-11-24 11:30:00.510614552+00:00   

                                     rtype  publisher_id  instrument_id  \
ts_recv                                                                   
2025-11-24 11:30:00.510234758+00:00     19            30     1275068417   
2025-11-24 11:30:00.510250326+00:00     19            30     1275068418   
2025-11-24 11:30:00.510278519+00:00     19            30     1409286145   

In [36]:
# Check GLBX.MDP3 dataset metadata (crypto futures)
print("Checking GLBX.MDP3 metadata...")
try:
    metadata = client.metadata.get_dataset_range(dataset='GLBX.MDP3')
    print(f"GLBX.MDP3 date range: {metadata}")
except Exception as e:
    print(f"Error getting metadata: {e}")

# Check what schemas are available
try:
    schemas = client.metadata.list_schemas(dataset='GLBX.MDP3')
    print(f"Available schemas for GLBX.MDP3: {schemas}")
except Exception as e:
    print(f"Error getting schemas: {e}")

Checking GLBX.MDP3 metadata...
GLBX.MDP3 date range: {'start': '2010-06-06T00:00:00.000000000Z', 'end': '2025-11-25T10:08:18.659189000Z', 'schema': {'mbo': {'start': '2017-05-21T00:00:00.000000000Z', 'end': '2025-11-25T10:08:18.659189000Z'}, 'mbp-1': {'start': '2010-06-06T00:00:00.000000000Z', 'end': '2025-11-25T10:08:18.659189000Z'}, 'mbp-10': {'start': '2010-06-06T00:00:00.000000000Z', 'end': '2025-11-25T10:08:18.659189000Z'}, 'tbbo': {'start': '2010-06-06T00:00:00.000000000Z', 'end': '2025-11-25T10:08:18.659189000Z'}, 'trades': {'start': '2010-06-06T00:00:00.000000000Z', 'end': '2025-11-25T10:08:18.659189000Z'}, 'bbo-1s': {'start': '2010-06-06T00:00:00.000000000Z', 'end': '2025-11-25T10:08:18.659189000Z'}, 'bbo-1m': {'start': '2010-06-06T00:00:00.000000000Z', 'end': '2025-11-25T10:08:18.659189000Z'}, 'ohlcv-1s': {'start': '2010-06-06T00:00:00.000000000Z', 'end': '2025-11-25T10:08:18.659189000Z'}, 'ohlcv-1m': {'start': '2010-06-06T00:00:00.000000000Z', 'end': '2025-11-25T10:08:18.659

In [11]:
# If data is available, parse options chain
if 'df' in locals() and not df.empty:
    # Parse option symbols
    # SPX options format: SPXYYMMDD[C|P]XXXXX
    # Where YYMMDD is expiration, C/P is call/put, XXXXX is strike
    
    def parse_option_symbol(symbol):
        if len(symbol) < 10:
            return None
        try:
            underlying = symbol[:3]  # SPX
            exp_date_str = symbol[3:9]  # YYMMDD
            option_type = symbol[9]  # C or P
            strike_str = symbol[10:]  # Strike price
            
            # Parse expiration
            exp_year = 2000 + int(exp_date_str[:2])
            exp_month = int(exp_date_str[2:4])
            exp_day = int(exp_date_str[4:6])
            expiration = datetime(exp_year, exp_month, exp_day).date()
            
            strike = float(strike_str) / 1000  # Strikes are in thousands
            
            return {
                'underlying': underlying,
                'expiration': expiration,
                'type': 'call' if option_type == 'C' else 'put',
                'strike': strike
            }
        except:
            return None
    
    # Apply parsing
    parsed = df['symbol'].apply(parse_option_symbol)
    valid_options = parsed.dropna()
    
    if not valid_options.empty:
        options_df = pd.DataFrame(list(valid_options))
        df_with_parsed = pd.concat([df.reset_index(drop=True), options_df.reset_index(drop=True)], axis=1)
        
        print(f"Parsed {len(options_df)} option contracts")
        print(options_df.head())
        
        # Get current SPX price for reference
        import yfinance as yf
        spx_price = yf.Ticker("^GSPC").fast_info['lastPrice']
        print(f"Current SPX price: {spx_price}")
        
        # Find near-the-money options
        current_date = datetime.now().date()
        future_options = options_df[options_df['expiration'] >= current_date]
        
        if not future_options.empty:
            # Find options expiring soon
            expirations = sorted(future_options['expiration'].unique())
            nearest_exp = expirations[0]
            near_exp_options = future_options[future_options['expiration'] == nearest_exp]
            
            # Find ATM strike
            atm_strike = near_exp_options.loc[(near_exp_options['strike'] - spx_price).abs().idxmin(), 'strike']
            
            atm_options = near_exp_options[near_exp_options['strike'] == atm_strike]
            
            print(f"\nNearest expiration: {nearest_exp}")
            print(f"ATM strike: {atm_strike}")
            print("ATM options:")
            print(atm_options)
            
            # Get prices from the data
            atm_data = df_with_parsed[
                (df_with_parsed['expiration'] == nearest_exp) & 
                (df_with_parsed['strike'] == atm_strike)
            ]
            
            if not atm_data.empty:
                latest_prices = atm_data.sort_values('ts_event').groupby('type').last()
                print("\nLatest prices:")
                print(latest_prices[['bid_price', 'ask_price', 'price']])
        
    else:
        print("No valid option symbols parsed")
        
else:
    print("No options data to parse")

No options data to parse


In [ ]:
# Test different schemas and symbols
schemas = ['mbp-1', 'ohlcv-1d', 'trades']
datasets_to_test = ['OPRA', 'CBOE', 'XNAS']

for dataset in datasets_to_test:
    for schema in schemas:
        try:
            test_data = client.timeseries.get_range(
                dataset=dataset,
                symbols=['SPX'],
                schema=schema,
                start=start_date,
                end=end_date,
                limit=10
            )
            df_test = test_data.to_df()
            if not df_test.empty:
                print(f"SUCCESS: {dataset} {schema} has {len(df_test)} SPX records")
                print(df_test.head(2))
                break
        except Exception as e:
            continue
    else:
        continue
    break

## Summary

After extensive testing, here's what we discovered about Databento for SPX options data:

### ✅ What Works
- **Databento client**: Successfully initialized and connected
- **OPRA.PILLAR dataset**: Available with comprehensive options data
- **Historical data**: Available back to 2013-04-01
- **Data retrieval**: Can successfully query and retrieve options trades data
- **Instrument definitions**: Available for mapping instrument_ids to symbols

### ❌ Key Limitation
- **SPX options not available**: The free tier only includes options on ETFs/stocks (SPY, TSLA, VSCO, etc.), but NOT index options like SPX
- **License requirement**: Live/recent data requires paid license after 2025-11-25T14:30:00Z

### 📊 Data Available in Free Tier
- SPY options (S&P 500 ETF proxy)
- Other equity/ETF options
- Historical data up to recent cutoff
- Full trade data with prices, sizes, timestamps

### 💡 Recommendations for Your Arbitrage Research

1. **For backtesting with proxy data**: Use SPY options + ES futures (your current approach)
2. **For true SPX data**: Consider upgrading to paid Databento plan or alternative providers
3. **Alternative data sources**:
   - Interactive Brokers TWS API (includes SPX options)
   - CBOE data feeds (may require partnership)
   - Other market data providers

The historical data extraction framework is ready - it just needs SPX options data to work with true put-call parity.

In [ ]:
# Complete Backtesting Setup (Ready for SPX Data)

# This setup is ready to use once you have access to SPX options data

def run_arbitrage_backtest(start_date, end_date, capital=1000000):
    """
    Run arbitrage backtest using historical options and futures data
    """
    print(f"Running arbitrage backtest from {start_date} to {end_date}")
    
    # Extract historical data
    backtest_data = extractor.extract_backtest_data(start_date, end_date)
    
    if backtest_data['options'].empty:
        print("No options data available for backtesting")
        return None
    
    # Initialize portfolio
    portfolio = {
        'cash': capital,
        'positions': {},
        'trades': [],
        'pnl': []
    }
    
    # Process each day
    for date in backtest_data['es_futures']['date'].unique():
        date = pd.to_datetime(date).date()
        print(f"Processing {date}")
        
        # Get data for this date
        day_options = backtest_data['options'][backtest_data['options']['date'] == date]
        day_futures = backtest_data['es_futures'][backtest_data['es_futures']['date'] == date]
        
        if day_options.empty or day_futures.empty:
            continue
            
        # Get futures price
        futures_price = day_futures.iloc[0]['close']
        
        # Find near-the-money options
        current_options = day_options[
            (day_options['expiration'] > date) &  # Not expired
            (abs(day_options['strike'] - futures_price) / futures_price < 0.05)  # Within 5% of spot
        ]
        
        if current_options.empty:
            continue
            
        # Calculate put-call parity
        calls = current_options[current_options['type'] == 'call']
        puts = current_options[current_options['type'] == 'put']
        
        if calls.empty or puts.empty:
            continue
            
        # Find matching strikes
        common_strikes = set(calls['strike']).intersection(set(puts['strike']))
        
        for strike in common_strikes:
            call_price = calls[calls['strike'] == strike]['price'].mean()
            put_price = puts[puts['strike'] == strike]['price'].mean()
            
            # Put-call parity: C - P = S - K * e^(-rt)
            # Simplified: C - P ≈ S - K (assuming no dividends, short time)
            parity_diff = call_price - put_price - (futures_price - strike)
            
            # Check for arbitrage opportunity
            threshold = 1.0  # $1 threshold
            
            if abs(parity_diff) > threshold:
                # Execute arbitrage trade
                trade_size = min(10, int(portfolio['cash'] / (futures_price * 100)))  # 100 multiplier
                
                if parity_diff > threshold:  # Calls overpriced
                    # Buy put, sell call, sell futures
                    trade = {
                        'date': date,
                        'type': 'arbitrage',
                        'action': 'buy_put_sell_call_sell_futures',
                        'strike': strike,
                        'size': trade_size,
                        'call_price': call_price,
                        'put_price': put_price,
                        'futures_price': futures_price,
                        'parity_diff': parity_diff
                    }
                    
                elif parity_diff < -threshold:  # Puts overpriced
                    # Buy call, sell put, buy futures
                    trade = {
                        'date': date,
                        'type': 'arbitrage',
                        'action': 'buy_call_sell_put_buy_futures',
                        'strike': strike,
                        'size': trade_size,
                        'call_price': call_price,
                        'put_price': put_price,
                        'futures_price': futures_price,
                        'parity_diff': parity_diff
                    }
                
                portfolio['trades'].append(trade)
                # Update cash (simplified - would need proper position tracking)
                portfolio['cash'] -= trade_size * abs(parity_diff) * 100  # Rough estimate
    
    # Calculate final results
    total_trades = len(portfolio['trades'])
    final_pnl = capital - portfolio['cash']
    
    results = {
        'total_trades': total_trades,
        'final_pnl': final_pnl,
        'win_rate': 0,  # Would need to calculate
        'sharpe_ratio': 0,  # Would need to calculate
        'max_drawdown': 0  # Would need to calculate
    }
    
    print(f"Backtest completed: {total_trades} trades, P&L: ${final_pnl:,.2f}")
    return results

# Example usage (would work with SPX data):
# results = run_arbitrage_backtest(
#     start_date=datetime(2024, 1, 1).date(),
#     end_date=datetime(2024, 12, 31).date(),
#     capital=1000000
# )

print("Arbitrage backtesting framework is ready!")
print("To use with real SPX data, you'll need:")
print("1. Paid Databento subscription for SPX options")
print("2. Or alternative data provider with SPX options")
print("3. Integration of the data into this framework")

In [41]:
# Check GLBX definitions to see what symbols are actually available
print("Checking GLBX.MDP3 definitions to find actual symbols...")

try:
    # Get definitions for recent period
    defs = client.timeseries.get_range(
        dataset='GLBX.MDP3',
        schema='definition',
        start=datetime(2025, 11, 24, 0, 0, 0),
        end=datetime(2025, 11, 25, 0, 0, 0),
        limit=50
    )

    df_defs = defs.to_df()
    print(f"GLBX definitions: {len(df_defs)} records")

    if not df_defs.empty:
        print("Columns:", df_defs.columns.tolist())

        # Check symbol column
        symbols = df_defs['symbol'].dropna().unique()
        print(f"Unique symbols: {len(symbols)}")
        print("Sample symbols:", symbols[:20])

        # Look for anything that might be crypto
        crypto_like = [s for s in symbols if any(term in str(s).upper() for term in ['BTC', 'ETH', 'BITCOIN', 'ETHEREUM', 'CRYPTO', 'DIGITAL'])]
        print(f"Crypto-like symbols: {crypto_like}")

        # Check underlying assets
        if 'underlying' in df_defs.columns:
            underlyings = df_defs['underlying'].dropna().unique()
            print(f"Underlying assets: {underlyings}")

        # Check for BRR, MET patterns
        brr_like = [s for s in symbols if 'BRR' in str(s) or 'MET' in str(s)]
        print(f"BRR/MET like symbols: {brr_like}")

except Exception as e:
    print(f"Error getting GLBX definitions: {e}")

print("\n" + "="*50)
print("CRYPTO ARBITRAGE SUMMARY:")
print("="*50)
print("📊 Available in Databento Free Tier:")
print("   ❌ CME Crypto Futures (GLBX.MDP3) - No crypto symbols found")
print("   ❌ Crypto Options - Not available")
print("")
print("🚀 Recommended Crypto Arbitrage Approaches:")
print("   1. Deribit BTC/ETH Options API")
print("   2. CME Crypto Futures (paid Databento)")
print("   3. DeFi Options (Lyra, Hegic)")
print("   4. Cross-exchange spot/futures arbitrage")
print("   5. Perpetual futures vs spot (Binance, Bybit, etc.)")
print("")
print("💡 For crypto options arbitrage:")
print("   - Deribit has BTC and ETH options")
print("   - Put-call parity possible with futures")
print("   - High volatility = good arbitrage potential")
print("   - Need real-time data feeds")

Checking GLBX.MDP3 definitions to find actual symbols...
GLBX definitions: 50 records
Columns: ['ts_event', 'rtype', 'publisher_id', 'instrument_id', 'raw_symbol', 'security_update_action', 'instrument_class', 'min_price_increment', 'display_factor', 'expiration', 'activation', 'high_limit_price', 'low_limit_price', 'max_price_variation', 'trading_reference_price', 'unit_of_measure_qty', 'min_price_increment_amount', 'price_ratio', 'inst_attrib_value', 'underlying_id', 'raw_instrument_id', 'market_depth_implied', 'market_depth', 'market_segment_id', 'max_trade_vol', 'min_lot_size', 'min_lot_size_block', 'min_lot_size_round_lot', 'min_trade_vol', 'contract_multiplier', 'decay_quantity', 'original_contract_size', 'trading_reference_date', 'appl_id', 'maturity_year', 'decay_start_date', 'channel_id', 'currency', 'settl_currency', 'secsubtype', 'group', 'exchange', 'asset', 'cfi', 'security_type', 'unit_of_measure', 'underlying', 'strike_price_currency', 'strike_price', 'match_algorithm', 

In [ ]:
# QUANDL TEST RESULTS
print("🧪 QUANDL FOR SPX OPTIONS ARBITRAGE")
print("="*45)

import quandl

print("✅ Quandl Status: Installed and working")
print(f"API Key: {'Configured' if quandl.ApiConfig.api_key else 'Not configured (free tier only)'}")

print("\n📊 AVAILABLE FREE DATASETS:")
free_datasets = [
    ('CBOE/VIX', 'VIX Volatility Index'),
    ('CBOE/SPX_PC', 'SPX Put-Call Ratio'),
    ('CBOE/VVIX', 'VIX of VIX'),
    ('WIKI/AAPL', 'Sample stock prices'),
]

for code, desc in free_datasets:
    print(f"• {code} - {desc}")

print("\n💰 PREMIUM DATASETS NEEDED:")
premium_datasets = [
    ('CBOE SPX options chains', 'Individual SPX option prices'),
    ('CME ES futures', 'E-mini S&P 500 futures'),
    ('Real-time options data', 'Live quotes and Greeks'),
]

for item, desc in premium_datasets:
    print(f"• {item} - {desc}")

print("\n" + "="*45)
print("🔍 INVESTIGATION SUMMARY:")
print("="*45)
print("✅ Databento: Free tier has OPRA options but no SPX")
print("✅ Quandl: Free tier has VIX/SPX_PC but no options chains")
print("❌ Both: Missing actual SPX options data for arbitrage")

print("\n💡 RECOMMENDED ALTERNATIVES:")
alternatives = [
    ('Polygon.io', 'Free tier + options API'),
    ('Alpha Vantage', 'Free API key + options'),
    ('Yahoo Finance', 'SPY options (free)'),
    ('Interactive Brokers', 'TWS API (free for small accounts)'),
    ('Deribit', 'Crypto options (free API)'),
    (' CME DataMine', 'Paid futures data'),
    ('CBOE Direct', 'Paid options data'),
]

for name, desc in alternatives:
    print(f"• {name}: {desc}")

print("\n🎯 NEXT STEPS:")
print("1. Get API key for preferred data source")
print("2. Test options data availability")
print("3. Implement put-call parity arbitrage")
print("4. Backtest with historical data")
print("5. Paper trade before live deployment")

print("\n📈 CURRENT STATUS:")
print("• SPY proxy arbitrage: ✅ Working")
print("• SPX options data: ❌ Not available in free tiers")
print("• Framework ready: ✅ HistoricalDataExtractor class")

In [22]:
    def get_instrument_definitions(self, date):
        """Get instrument definitions for a date to map instrument_id to symbols"""
        cache_key = date
        if cache_key in self.instrument_cache:
            return self.instrument_cache[cache_key]
            
        try:
            # Get definitions for the date (start from midnight)
            start_time = datetime.combine(date, datetime.min.time())
            end_time = datetime.combine(date + timedelta(days=1), datetime.min.time())
            
            print(f"Getting definitions from {start_time} to {end_time}")
            
            definitions = self.client.timeseries.get_range(
                dataset='OPRA.PILLAR',
                schema='definition',
                start=start_time,
                end=end_time,
                limit=1000  # Start with smaller limit
            )
            
            df_def = definitions.to_df()
            print(f"Definitions query returned {len(df_def)} rows")
            
            if not df_def.empty:
                print("Definitions columns:", df_def.columns.tolist())
                if 'symbol' in df_def.columns:
                    spx_symbols = df_def[df_def['symbol'].str.startswith('SPX', na=False)]
                    print(f"Found {len(spx_symbols)} SPX symbols in definitions")
                    
                    # Create mapping from instrument_id to symbol info
                    instrument_map = {}
                    for _, row in spx_symbols.iterrows():
                        symbol = row['symbol']
                        parsed = self.parse_option_symbol(symbol)
                        if parsed:
                            instrument_map[row['instrument_id']] = parsed
                    
                    print(f"Created mapping for {len(instrument_map)} instruments")
                    self.instrument_cache[cache_key] = instrument_map
                    return instrument_map
                else:
                    print("No 'symbol' column in definitions")
            else:
                print(f"No definitions data returned")
                return {}
                
        except Exception as e:
            print(f"Error getting definitions for {date}: {e}")
            return {}

In [23]:
# Test the historical data extraction
print("Testing historical data extraction...")

# Test with a recent date
test_date = datetime(2025, 11, 24).date()
print(f"Testing extraction for {test_date}")

# Extract data for one day
backtest_data = extractor.extract_backtest_data(
    start_date=test_date,
    end_date=test_date,
    sample_days=None
)

print("\nExtraction Results:")
print(f"Options data shape: {backtest_data['options'].shape}")
print(f"ES futures data shape: {backtest_data['es_futures'].shape}")

if not backtest_data['options'].empty:
    print("\nSample options data:")
    print(backtest_data['options'].head())
    
    # Show some statistics
    print(f"\nUnique expirations: {backtest_data['options']['expiration'].nunique()}")
    print(f"Strike range: {backtest_data['options']['strike'].min()} - {backtest_data['options']['strike'].max()}")
    print(f"Calls: {len(backtest_data['options'][backtest_data['options']['type'] == 'call'])}")
    print(f"Puts: {len(backtest_data['options'][backtest_data['options']['type'] == 'put'])}")

if not backtest_data['es_futures'].empty:
    print("\nES futures data:")
    print(backtest_data['es_futures'])

Testing historical data extraction...
Testing extraction for 2025-11-24
Extracting backtest data from 2025-11-24 to 2025-11-24
Processing 2025-11-24
Extracting SPX options for 2025-11-24
No instrument definitions for 2025-11-24

Extraction Results:
Options data shape: (0, 0)
ES futures data shape: (1, 6)

ES futures data:
         date     open     high     low    close   volume
0  2025-11-24  6654.75  6731.75  6625.0  6721.25  2592859


In [38]:
# Check for crypto options datasets
print("Checking for crypto options datasets...")

datasets = client.metadata.list_datasets()
crypto_options_datasets = [ds for ds in datasets if any(term in ds.upper() for term in ['CRYPTO', 'BTC', 'ETH', 'OPTION'])]

print(f"Potential crypto/options datasets: {crypto_options_datasets}")

# Check if any datasets might have crypto options
for ds in ['OPRA.PILLAR', 'GLBX.MDP3', 'DBEQ.BASIC']:
    try:
        print(f"\nChecking {ds} for crypto symbols...")
        # Get recent definitions
        defs = client.timeseries.get_range(
            dataset=ds,
            schema='definition',
            start=datetime(2025, 11, 24, 0, 0, 0),
            end=datetime(2025, 11, 25, 0, 0, 0),
            limit=100
        )

        df_defs = defs.to_df()
        if not df_defs.empty:
            symbols = df_defs['symbol'].dropna().unique()
            crypto_symbols = [s for s in symbols if any(term in str(s).upper() for term in ['BTC', 'ETH', 'CRYPTO'])]

            if crypto_symbols:
                print(f"Found crypto symbols in {ds}: {crypto_symbols[:5]}")
            else:
                print(f"No obvious crypto symbols in {ds}")

    except Exception as e:
        print(f"Error checking {ds}: {e}")

print("\n" + "="*50)
print("CRYPTO ARBITRAGE OPPORTUNITIES:")
print("="*50)
print("1. CME Crypto Futures (GLBX.MDP3): BTC, ETH futures")
print("2. DeFi Options Platforms:")
print("   - Lyra (Optimism): BTC, ETH options")
print("   - Hegic: Various crypto options")
print("   - Premia: DeFi options")
print("3. Centralized Exchanges:")
print("   - CME: BTC/ETH futures & options")
print("   - Deribit: BTC/ETH options")
print("   - Binance/Coinbase: Limited options")
print("4. Cross-exchange arbitrage:")
print("   - Spot vs Futures")
print("   - Futures vs Options")
print("   - Exchange rate differences")

Checking for crypto options datasets...
Potential crypto/options datasets: []

Checking OPRA.PILLAR for crypto symbols...
Potential crypto/options datasets: []

Checking OPRA.PILLAR for crypto symbols...
No obvious crypto symbols in OPRA.PILLAR

Checking GLBX.MDP3 for crypto symbols...
No obvious crypto symbols in OPRA.PILLAR

Checking GLBX.MDP3 for crypto symbols...
No obvious crypto symbols in GLBX.MDP3

Checking DBEQ.BASIC for crypto symbols...
No obvious crypto symbols in GLBX.MDP3

Checking DBEQ.BASIC for crypto symbols...
No obvious crypto symbols in DBEQ.BASIC

CRYPTO ARBITRAGE OPPORTUNITIES:
1. CME Crypto Futures (GLBX.MDP3): BTC, ETH futures
2. DeFi Options Platforms:
   - Lyra (Optimism): BTC, ETH options
   - Hegic: Various crypto options
   - Premia: DeFi options
3. Centralized Exchanges:
   - CME: BTC/ETH futures & options
   - Deribit: BTC/ETH options
   - Binance/Coinbase: Limited options
4. Cross-exchange arbitrage:
   - Spot vs Futures
   - Futures vs Options
   - Exch